# 🪵 Snowflake Trail - Step 3: Custom Logging

This notebooks shows how to set up custom event logging in addition to the standard event-logs from Tasks, Dynamic Tables, etc.

We will set up event logs for 3 common events that users want to be notified about:
* Stale Streams
* Streams stale in <24 hours
* Overloaded Warehouse (queued queries >10%)

## 3.0. Custom Loggers for error, warning and info
* we can call these functions to log events
* ⚠️ since the logger-functions are inside the SNOWTRAIL_DEMO database, the events will be logged to the event-table set for this database

In [ ]:
--- function to manually log error to event_table
create or replace function SNOWTRAIL_DEMO.OBSERV.ERROR_LOG(MESSAGE varchar)
returns VARCHAR
language PYTHON
RUNTIME_VERSION = 3.8
HANDLER = 'run'
as $$
import logging
logger = logging.getLogger("Snowtrail_logger")

def run(MESSAGE):
  logger.error(MESSAGE)
  return "Pipeline Error Logged"
$$;

In [ ]:
--- function to manually log warning to event_table
create or replace function SNOWTRAIL_DEMO.OBSERV.WARN_LOG(MESSAGE varchar)
returns VARCHAR
language PYTHON
RUNTIME_VERSION = 3.8
HANDLER = 'run'
as $$
import logging
logger = logging.getLogger("Snowtrail_logger")

def run(MESSAGE):
  logger.warn(MESSAGE)
  return "Pipeline Warning Logged"
$$;

In [ ]:
--- function to manually log warning to event_table
create or replace function SNOWTRAIL_DEMO.OBSERV.INFO_LOG(MESSAGE varchar)
returns VARCHAR
language PYTHON
RUNTIME_VERSION = 3.8
HANDLER = 'run'
as $$
import logging
logger = logging.getLogger("Snowtrail_logger")

def run(MESSAGE):
  logger.info(MESSAGE)
  return "Pipeline Info Logged"
$$;

## 3.1. Custom Alert on Streams
* logging new stale streams as errors
* logging streams going stale within 24h as warning


In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.STALE_STREAMS_TO_EVENT_TABLE
schedule = '60 minute'
comment = 'checks all Streams in database'
if (exists(
    with 
        GET_STALE_STREAMS as procedure()
        returns table()
        language SQL
        as
        $$
        begin    
            show streams in database;
            let result resultset := (  
                select 
                    concat($3,'.',$4,'.',$2) as STREAM_FULL_NAME
                from 
                    table(result_scan(last_query_id()))
                where
                    $11 = 'true'   ---is stale
                    and $13 >= SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME()    -- new since the last Alert run (system function)
            );
            return table(result);
        end;
        $$
    call GET_STALE_STREAMS()
))

then
    begin
        let STALE_STREAMS resultset := (select * from table(result_scan(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID())));
        
        for RECORD in STALE_STREAMS do    
            let ERROR_MESSAGE string := ('Stream '||RECORD.STREAM_FULL_NAME||' is stale.');
              
            select SNOWTRAIL_DEMO.OBSERV.ERROR_LOG(:ERROR_MESSAGE);
        end for;
end; 

In [ ]:
alter alert SNOWTRAIL_DEMO.OBSERV.STALE_STREAMS_TO_EVENT_TABLE resume;

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.STREAMS_WARNING_TO_EVENT_TABLE
schedule = '60 minute'
comment = 'checks all Streams in database'
if (exists(
    with 
        OLD_STREAMS as procedure()
        returns table()
        language SQL
        as
        $$
        begin    
            show streams in database;
            
            let result resultset := (  
                select 
                    concat($3,'.',$4,'.',$2) as STREAM_FULL_NAME,
                    timediff(hour, current_timestamp, $13) as STALE_IN_HOURS
                from 
                    table(result_scan(last_query_id()))
                where
                    $11 = 'false'   ---is not yet stale
                    and STALE_IN_HOURS < 24
            );
            return table(result);
        end;
        $$    
    call OLD_STREAMS()
))

then
    begin
        let ALMOST_STALE_STREAMS resultset := (select * from table(result_scan(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID())));
        
        for RECORD in ALMOST_STALE_STREAMS do    
            let WARN_MESSAGE string := ('Stream '||RECORD.STREAM_FULL_NAME||' will become stale in '||RECORD.STALE_IN_HOURS||' hours.');
              
            select SNOWTRAIL_DEMO.OBSERV.WARN_LOG(:WARN_MESSAGE);
        end for;
end; 

In [ ]:
alter alert SNOWTRAIL_DEMO.OBSERV.STREAMS_WARNING_TO_EVENT_TABLE resume;

## 3.2. Custom Alert on Warehouse overload

Checking for all Warehouses in the account.

In [ ]:
-- see https://docs.snowflake.com/en/sql-reference/functions/warehouse_load_history
 select
    WAREHOUSE_NAME,
    START_TIME,
    AVG_QUEUED_LOAD
from 
    table(SNOWFLAKE.INFORMATION_SCHEMA.WAREHOUSE_LOAD_HISTORY(
        DATE_RANGE_START => timeadd(hour, -5, current_timestamp)
    ))
where
    AVG_QUEUED_LOAD > 0.9
    --AVG_QUEUED_LOAD < 0.9     -- to test for results if you don't have warehouses with >90% utilization
order by
    START_TIME desc
;

In [ ]:
-- hourly check on WAREHOUSE_LOAD_HISTORY for queued up queries

create or replace alert SNOWTRAIL_DEMO.OBSERV.WAREHOUSE_LOAD_WARN_TO_EVENT_TABLE
schedule = '60 minute'
comment = 'checks Warehouses in this database for overload and logs them as warning to the event table'
if (exists(
    select 
        distinct WAREHOUSE_NAME
    from 
        table(SNOWFLAKE.INFORMATION_SCHEMA.WAREHOUSE_LOAD_HISTORY(
            DATE_RANGE_START => SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME()
        ))
    where
        AVG_QUEUED_LOAD > 0.9
        and WAREHOUSE_NAME not like 'COMPUTE_SERVICE_WH%'       -- exclude serverless
    ))
then
    begin
        let WH_OVERLOAD resultset := (select * from table(result_scan(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID())));
        
        for RECORD in WH_OVERLOAD do    
            let WARN_MESSAGE string := ('Warehouse '||RECORD.WAREHOUSE_NAME||' was queued up between '||SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME()||' and '||current_timestamp()||'.');
              
            select SNOWTRAIL_DEMO.OBSERV.WARN_LOG(:WARN_MESSAGE);
        end for;
    end; 
;


In [ ]:
alter alert SNOWTRAIL_DEMO.OBSERV.WAREHOUSE_LOAD_WARN_TO_EVENT_TABLE resume;

...these types of triggers are not so easy to test. But you can still check the Alert_History to see if all alerts ran correctly.